In [1]:
ls

chuvash-corpus-parallel.zip  corpus.ipynb          data/
chv_rus_1M_Corpus_data/      corpus_test.csv
chv_rus_1M_Corpus_model/     corpus_train_all.csv


In [2]:
import sockeye
import tensorboard

In [3]:
import torch
print(torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(torch.cuda.get_device_name(i))

/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1
Tesla V100-SXM2-32GB


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datasets import load_dataset

In [5]:
src = 'chv'
tgt = 'rus'
name = '1M_Corpus'

In [5]:
import sockeye
import tensorboard

print(tensorboard.__file__)
print(sockeye.__file__)

/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/tensorboard/__init__.py
/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/__init__.py


In [11]:
train_path = 'corpus_train_all.csv'
test_path = 'corpus_test.csv'

df_test = pd.read_csv(test_path)
df_train = load_dataset('csv', data_files=train_path).shuffle(seed=42)

split_dataset = df_train['train'].train_test_split(test_size=0.1)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 26.80it/s]
Generating train split: 0 examples [00:00, ? examples/s]/home/kvzhirnov/.conda/envs/diploma/lib/python3.11/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/kvzhirnov/.cache/huggingface/datasets/csv/default-bc85784d1dc8f6ce/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 22.47it/s]


In [13]:
!mkdir 'data'
!mkdir 'data/train'
!mkdir 'data/dev'
!mkdir 'data/test'

In [14]:
def writelines(list, filename):
  with open(filename, 'w', encoding='utf-8') as file:
    for item in list:
      file.write('%s\n' % item)

chv_train = [l['chv'] for l in split_dataset['train']]
rus_train = [l['rus'] for l in split_dataset['train']]

chv_dev = [l['chv'] for l in split_dataset['test']]
rus_dev = [l['rus'] for l in split_dataset['test']]

chv_test = df_test['chv'].tolist()
rus_test = df_test['rus'].tolist()

writelines(chv_train, 'data/train/train.chv.txt')
writelines(rus_train, 'data/train/train.rus.txt')

writelines(chv_dev, 'data/dev/dev.chv.txt')
writelines(rus_dev, 'data/dev/dev.rus.txt')

writelines(chv_test, 'data/test/test.chv.txt')
writelines(rus_test, 'data/test/test.rus.txt')

In [15]:
file_name = 'data/train/train.chv.txt'
number_of_rows = 5  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

Хӑйӗн чунӗ мӗн пӗлнине нумайӑшне пӗлмест вӑл».
Ҫӗнӗ ят тухать халӑха: Ленин.
Этем телейлӗ пулассишӗн тӑрӑшать, апла-тӑк вӑл саккунлӑ япала.
Анчах нумайтарах ыйтмастпӑр-и-ха эпир энтомологсем хушшинчи чи хастар ҫынран?
Наташа калаҫма чарӑнсан, Павел ура ҫине тӑрса лӑпкӑн ыйтрӗ:


In [16]:
file_name = 'data/train/train.rus.txt'
number_of_rows = 5  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

Она многого не знает из того, что знает душа ее.
Появилось новое имя: Ленин.
В человека вложена потребность счастия; стало быть, она законна.
Но не слишком ли многого мы требуем от самого большого энтузиаста среди энтомологов?
Когда Наташа замолчала, встал Павел и спокойно спросил:


In [19]:
!python subword-nmt/subword_nmt/learn_joint_bpe_and_vocab.py --input data/train/train.{src}.txt data/train/train.{tgt}.txt  -s 10000 -o data/bpe.codes  --write-vocabulary data/bpe.vocab.{src} data/bpe.vocab.{tgt}

100%|#####################################| 10000/10000 [02:17<00:00, 72.47it/s]


In [20]:
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{src} --vocabulary-threshold 50 < data/train/train.{src}.txt > data/train/train.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data/train/train.{tgt}.txt > data/train/train.{tgt}.bpe

!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{src} --vocabulary-threshold 50 < data/dev/dev.{src}.txt > data/dev/dev.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data/dev/dev.{tgt}.txt > data/dev/dev.{tgt}.bpe

!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{src} --vocabulary-threshold 50 < data/test/test.{src}.txt > data/test/test.{src}.bpe
!python subword-nmt/subword_nmt/apply_bpe.py -c data/bpe.codes --vocabulary data/bpe.vocab.{tgt} --vocabulary-threshold 50 < data/test/test.{tgt}.txt > data/test/test.{tgt}.bpe

In [2]:
!python -m sockeye.prepare_data -s data/train/train.{src}.bpe -t data/train/train.{tgt}.bpe --shared-vocab -o {src}_{tgt}_{name}_data

usage: prepare_data.py [-h] [--config CONFIG] --source SOURCE
                       [--source-factors SOURCE_FACTORS [SOURCE_FACTORS ...]]
                       [--source-factors-use-source-vocab SOURCE_FACTORS_USE_SOURCE_VOCAB [SOURCE_FACTORS_USE_SOURCE_VOCAB ...]]
                       [--target-factors TARGET_FACTORS [TARGET_FACTORS ...]]
                       [--target-factors-use-target-vocab TARGET_FACTORS_USE_TARGET_VOCAB [TARGET_FACTORS_USE_TARGET_VOCAB ...]]
                       --target TARGET
                       [--end-of-prepending-tag END_OF_PREPENDING_TAG]
                       [--source-vocab SOURCE_VOCAB]
                       [--target-vocab TARGET_VOCAB]
                       [--source-factor-vocabs SOURCE_FACTOR_VOCABS [SOURCE_FACTOR_VOCABS ...]]
                       [--target-factor-vocabs TARGET_FACTOR_VOCABS [TARGET_FACTOR_VOCABS ...]]
                       [--shared-vocab] [--num-words NUM_WORDS]
                       [--word-min-count WORD_MIN_CO

In [6]:
!python -m sockeye.train --help

usage: train.py [-h] [--config CONFIG] [--source SOURCE]
                [--source-factors SOURCE_FACTORS [SOURCE_FACTORS ...]]
                [--source-factors-use-source-vocab SOURCE_FACTORS_USE_SOURCE_VOCAB [SOURCE_FACTORS_USE_SOURCE_VOCAB ...]]
                [--target-factors TARGET_FACTORS [TARGET_FACTORS ...]]
                [--target-factors-use-target-vocab TARGET_FACTORS_USE_TARGET_VOCAB [TARGET_FACTORS_USE_TARGET_VOCAB ...]]
                [--target TARGET]
                [--end-of-prepending-tag END_OF_PREPENDING_TAG]
                [--prepared-data PREPARED_DATA] --validation-source
                VALIDATION_SOURCE
                [--validation-source-factors VALIDATION_SOURCE_FACTORS [VALIDATION_SOURCE_FACTORS ...]]
                --validation-target VALIDATION_TARGET
                [--validation-target-factors VALIDATION_TARGET_FACTORS [VALIDATION_TARGET_FACTORS ...]]
                [--no-bucketing] [--bucket-width BUCKET_WIDTH]
                [--bucket-scalin

In [6]:
!python -m sockeye.train -d {src}_{tgt}_{name}_data -vs data/dev/dev.{src}.bpe -vt data/dev/dev.{tgt}.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o {src}_{tgt}_{name}_model --num-layers 6 --keep-last-params 3 --batch-size 2048 --optimized-metric bleu --max-num-checkpoint-not-improved 5 --amp

[INFO:sockeye.utils] Sockeye: 3.1.34, commit unknown, path /home/kvzhirnov/EXP_ONLY_CORPUS/sockeye/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1 (/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /home/kvzhirnov/EXP_ONLY_CORPUS/sockeye/sockeye/train.py -d chv_rus_1M_Corpus_data -vs data/dev/dev.chv.bpe -vt data/dev/dev.rus.bpe --encoder transformer --decoder transformer --transformer-model-size 512 --transformer-feed-forward-num-hidden 2048 --transformer-dropout-prepost 0.1 --num-embed 512 --max-seq-len 100 --decode-and-evaluate 500 -o chv_rus_1M_Corpus_model --num-layers 6 --keep-last-params 3 --batch-size 2048 --optimized-metric bleu --max-num-checkpoint-not-improved 5 --amp
[INFO:sockeye.utils] Arguments: Namespace(config=None, source=None, source_factors=[], source_factors_use_source_vocab=[], target_factors=[], target_factors_use_target_vocab=[], target=None, end_of_prepending_tag=None, prepared_data='chv_ru

In [7]:
!python -m sockeye.translate -m {src}_{tgt}_{name}_model -i data/test/test.{src}.bpe -o data/test/test.{tgt}.pred.bpe --batch-size 128 --beam-size 5

^C
Traceback (most recent call last):
  File "/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/translate.py", line 25, in <module>
    from sockeye.lexicon import load_restrict_lexicon, RestrictLexicon
  File "/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/lexicon.py", line 27, in <module>
    from sockeye.data_io import SequenceReader
  File "/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/data_io.py", line 32, in <module>
    import torch
  File "/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/torch/__init__.py", line 895, in <module>
    import torch.utils.data
  File "/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/t

In [9]:
!sed -r 's/(@@ )|(@@ ?$)//g' data/test/test.{tgt}.pred.bpe > data/test/test.{tgt}.pred.txt

In [10]:
!sacrebleu data/test/test.rus.txt -tok none -i data/test/test.{tgt}.pred.txt

{
 "name": "BLEU",
 "score": 18.4,
 "signature": "nrefs:1|case:mixed|eff:no|tok:none|smooth:exp|version:2.3.1",
 "verbose_score": "46.6/24.3/14.3/8.8 (BP = 0.947 ratio = 0.949 hyp_len = 1032354 ref_len = 1088181)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "none",
 "smooth": "exp",
 "version": "2.3.1"
}


In [1]:
file_name = 'data/test/test.chv.txt'
number_of_rows = 10  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

Ывӑнса халтан кайман пулсан, чӗнсем каснипе шыҫса тухнисем пулман пулсан (ҫитменнине манӑн тата грыжа пур), эпӗ, тен, хамӑрӑн йывӑр походӑн кунӗсене татах та пурӑнса ирттерме ӗмӗтленнӗ пулӑттӑм.
— Акӑ мӗнле официально паллаштаратӑн эсӗ хӑвпа!
Эпир ҫул ҫине тӑсӑлса выртрӑмӑр та ракетӑсем ҫине пӑхатпӑр.
Пусма каштисен витӗр нумай-нумай клепкӑллӑ металл сфера сийӗ йӑлтӑртатать.
Ҫак кунсенче каҫхине эсир пире петӗр!
— Епле иртнӗ-ха эпир унтан, тӗлӗнмелле, — терӗ Заикин.
8. Моисей хӑйӗн хуньӑшне Ҫӳлхуҫа фараонпа [мӗнпур] Египет халӑхне Израильшӗн мӗн-мӗн кӑтартни ҫинчен, ҫул ҫинче хӑйсене тӗл пулнӑ мӗнпур йывӑрлӑх ҫинчен тата Ҫӳлхуҫа вӗсене [фараон аллинчен, Египет ҫыннисен аллинчен] епле хӑтарни ҫинчен каласа панӑ.
Вара Таракуль пулемётне пӑрахнӑ та йӑлтах вӑйсӑрланса ҫитнӗ юлташне тӗрткелесе:— Хамӑрӑннисем! Хамӑрӑннисем! Хамӑрӑннисем! — тесе вӗҫӗмсӗр кӑшкӑрнӑ.
Хӗрарӑмсем пӑшӑрханса ӳкрӗҫ.
Ӑна ҫак янравлӑн илтӗнекен сӑмах ытла та килӗшнӗ пулас.


In [2]:
file_name = 'data/test/test.rus.txt'
number_of_rows = 10  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

И если бы не усталость, если бы не потертости, нарывы (а у меня грыжа), я, пожалуй, снова пожелал бы пережить дни нашего немыслимого похода.
— Вот как официально ты представляешься!
Мы падали на дорогу и следили за ними.
Сквозь балки лесов поблёскивала металлическая, с частой клёпкой, поверхность сферического тела.
Вечером в нас стреляли.
— Как мы прошли, ума не приложу, — сказал Заикин.
8. И рассказал Моисей тестю своему о всем, что сделал Господь с фараоном и со [всеми] Египтянами за Израиля, и о всех трудностях, какие встретили их на пути, и как избавил их Господь [из руки фараона и из руки Египтян].
Тогда он бросил пулемёт, стал трясти совсем ослабевшего друга, крича ему только одно слово:— Наши, наши, наши!
Женщины забеспокоились.
Которому, видимо, очень нравилось звучное название.


In [3]:
file_name = 'data/test/test.rus.pred.txt'
number_of_rows = 10  # The number of rows you want to read

with open(file_name, 'r') as file:
    for i in range(number_of_rows):
        line = file.readline()
        print(line.strip())

Если бы не усталость, если бы не распухшие от ремней (к тому же у меня была грыжа), я, может быть, мечтал бы еще и еще раз прожить дни нашего тяжелого похода.
— Вот как официально ты познакомишь с собой!
Мы растянулись на дороге и смотрели на ракеты.
Сквозь перекладины сверкала металлическая сфера с многочисленными клепками.
Вечером этих дней вы стреляли в нас!
— Как мы прошли, странно, — сказал Заикин.
8. И рассказал Моисей тестю своему о том, что сделал Господь с фараоном [все] Египтянами для Израиля, и о всех трудностях, которые встречались им на пути, и как избавил их Господь [из руки фараона и из руки Египтян].
Тогда Таракуль бросил пулемет и, подталкивая ослабевшего товарища, все кричал: — Свои! Свои! Наши!
Женщины заволновались.
Ему, видимо, очень нравилось это звучное слово.


In [8]:
!python -m sockeye.translate -m {src}_{tgt}_{name}_model -i data/test/test_replacement_50.{src}.bpe -o data/test/test_replacement_50.{tgt}.pred.bpe --batch-size 128 --beam-size 5

[INFO:sockeye.utils] Sockeye: 3.1.34, commit 4c30942ddb523533bccb4d2cbb3e894e45b1db93, path /home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/__init__.py
[INFO:sockeye.utils] PyTorch: 1.13.1 (/home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/torch/__init__.py)
[INFO:sockeye.utils] Command: /home/kvzhirnov/.conda/envs/diploma/lib/python3.9/site-packages/sockeye/translate.py -m chv_rus_1M_Corpus_model -i data/test/test_replacement_50.chv.bpe -o data/test/test_replacement_50.rus.pred.bpe --batch-size 128 --beam-size 5
[INFO:sockeye.utils] Arguments: Namespace(config=None, input='data/test/test_replacement_50.chv.bpe', input_factors=None, json_input=False, output='data/test/test_replacement_50.rus.pred.bpe', models=['chv_rus_1M_Corpus_model'], checkpoints=None, nbest_size=1, beam_size=5, greedy=False, beam_search_stop='all', batch_size=128, chunk_size=None, sample=None, seed=None, ensemble_mode='linear', bucket_width=10, max_input_length=None, max_output

In [10]:
!sed -r 's/(@@ )|(@@ ?$)//g' data/test/test_replacement_50.{tgt}.pred.bpe > data/test/test_replacement_50.{tgt}.pred.txt

In [12]:
!sacrebleu data/test/test_replacement_50.rus.txt -tok none -i data/test/test_replacement_50.{tgt}.pred.txt

{
 "name": "BLEU",
 "score": 6.1,
 "signature": "nrefs:1|case:mixed|eff:no|tok:none|smooth:exp|version:2.3.1",
 "verbose_score": "28.8/9.4/3.9/1.7 (BP = 0.942 ratio = 0.943 hyp_len = 1026909 ref_len = 1088642)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "none",
 "smooth": "exp",
 "version": "2.3.1"
}


In [10]:
pip install tensorflow tensorboard

Note: you may need to restart the kernel to use updated packages.


In [12]:
!tensorboard dev upload --logdir

2023-05-17 20:48:13.436031: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: tensorboard dev upload [-h] [--helpfull] [--logdir PATH] [--name NAME]
                              [--description DESCRIPTION] [--verbose VERBOSE]
                              [--dry_run] [--one_shot] [--plugins PLUGINS]
tensorboard dev upload: error: argument --logdir: expected one argument
